# HD HomeRun Analysis

---

Info: https://www.silicondust.com/hdhomerun/hdhomerun_development.pdf

Did not work:
- pip install pyhdhomerun
- conda install pyhdhomerun
- pip install git+git://github.com/burnnat/hdhr

---

In [4]:
pip install hdhr

Note: you may need to restart the kernel to use updated packages.


In [5]:
import hdhr

In [6]:
help(hdhr)

Help on package hdhr:

NAME
    hdhr

PACKAGE CONTENTS
    adapter
    adapter_types
    constants
    externals
    libhdhr
    log_config
    types
    utility

FILE
    /Users/burkesquires/anaconda3/lib/python3.7/site-packages/hdhr/__init__.py




In [7]:
hdhr

<module 'hdhr' from '/Users/burkesquires/anaconda3/lib/python3.7/site-packages/hdhr/__init__.py'>

In [8]:
#!/usr/bin/env python3

from sys import exit
from pprint import pprint
from os import chdir

from hdhr.adapter import HdhrUtility, HdhrDeviceQuery
from hdhr.constants import MAP_US_BCAST

def find_devices():
    devices = HdhrUtility.discover_find_devices_custom()
    
    for device in devices:
        print("Found: %s" % (device))

    return devices

def set_tuner_vchannel(device_adapter, vchannel):
    
    device_adapter.set_tuner_vchannel(vchannel)

    (vstatus, raw_data) = device_adapter.get_tuner_vstatus()
    print(vstatus)

def set_stream(device_adapter, vchannel, target_uri):

    device_adapter.set_tuner_vchannel(vchannel)
    device_adapter.set_tuner_target(target_uri)

def get_supported(device_adapter):

    pprint(device_adapter.get_supported())

def scan(device_adapter):

    def found(result, scan_info):
        result.dump_programs()

    def progress_update(scan_info, progress):
        print("Progress: %.2f  Scanned= (%d)" % (progress, 
                                                 scan_info.scanned_channels))

    device_adapter.scan_channels(MAP_US_CABLE, found_cb=found, 
                                 incremental_cb=progress_update)

def get_count():

    return HdhrUtility.get_channels_in_range(MAP_US_CABLE)

devices = find_devices()

if not devices:
    print("Could not find any devices.")
    exit()

i = 0
for device in devices:
    print("%d: %s" % (i, device))

    hd = HdhrUtility.device_create_from_str(device.nice_device_id)
    device_adapter = HdhrDeviceQuery(hd)

    hw_model = device_adapter.get_hw_model_str()
    print("HW Model: %s" % (hw_model))

    model = device_adapter.get_model_str()
    print("Model: %s" % (model))

    version = device_adapter.get_version()
    print("Version: %s" % (version))

    features = device_adapter.get_supported()
    print("Features: %s" % (features))

    i += 1
    
print

for tuner in range(0, devices[0].tuner_count):
    first_device_str = ("%s-%d" % (devices[0].nice_device_id, tuner))

    hd = HdhrUtility.device_create_from_str(first_device_str)

    device_adapter = HdhrDeviceQuery(hd)

    (status, raw_data) = device_adapter.get_tuner_status()
    print("Status: %s" % (status))

    (vstatus, raw_data) = device_adapter.get_tuner_vstatus()
    print("Vstatus: %s" % (vstatus))

    streaminfo = device_adapter.get_tuner_streaminfo()
    print("Streaminfo: %s" % (streaminfo))

    program = device_adapter.get_tuner_program()
    print("Program: %s" % (program))

#device_adapter.set_tuner_vchannel(49)
#device_adapter.set_tuner_target('rtp://192.168.5.13:7891')
#device_adapter.set_tuner_target(None)

#get_supported(device_adapter)

#print get_count()

#scan(device_adapter)

print("Done.")

OSError: dlopen(libhdhomerun.so, 6): image not found

In [41]:
import hdhr
hdhr

<module 'hdhr' from '/Users/burkesquires/anaconda3/lib/python3.7/site-packages/hdhr/__init__.py'>

Download and `make` software here:

%%bash
wget https://github.com/Silicondust/libhdhomerun .
cd libhdhomerun
make

    burkesquires@Burkes-Mac-mini hdhomerun_config_gui % hdhomerun_config 
    Usage:
        hdhomerun_config discover
        hdhomerun_config <id> get help
        hdhomerun_config <id> get <item>
        hdhomerun_config <id> set <item> <value>
        hdhomerun_config <id> scan <tuner> [<filename>]
        hdhomerun_config <id> save <tuner> <filename>
        hdhomerun_config <id> upgrade <filename>
    burkesquires@Burkes-Mac-mini hdhomerun_config_gui % hdhomerun_config discover
    hdhomerun device 10758528 found at 192.168.1.43


Discover any HDHomeRun devices on the network

In [26]:
contents = !hdhomerun_config discover
hdhr_raw = contents.get_list()
hdhr_info = hdhr_raw[0].split()
hdhr_id = hdhr_info[2]
hdhr_ip = hdhr_info[5]

'10758528'

In [30]:
!hdhomerun_config $hdhr_id

Usage:
	hdhomerun_config discover
	hdhomerun_config <id> get help
	hdhomerun_config <id> get <item>
	hdhomerun_config <id> set <item> <value>
	hdhomerun_config <id> scan <tuner> [<filename>]
	hdhomerun_config <id> save <tuner> <filename>
	hdhomerun_config <id> upgrade <filename>


In [27]:
!hdhomerun_config $hdhr_id get help

Supported configuration options:
/lineup/scan
/sys/copyright
/sys/debug
/sys/features
/sys/hwmodel
/sys/model
/sys/restart <resource>
/sys/version
/tuner<n>/channel <modulation>:<freq|ch>
/tuner<n>/channelmap <channelmap>
/tuner<n>/debug
/tuner<n>/filter "0x<nnnn>-0x<nnnn> [...]"
/tuner<n>/lockkey
/tuner<n>/program <program number>
/tuner<n>/status
/tuner<n>/streaminfo
/tuner<n>/target <ip>:<port>
/tuner<n>/vchannel <vchannel>



In [32]:
!hdhomerun_config $hdhr_id get /sys/hwmodel

HDHR5-4US


In [33]:
!hdhomerun_config $hdhr_id get /sys/features

channelmap: us-bcast us-cable us-hrc us-irc kr-bcast kr-cable
modulation: 8vsb qam256 qam64
auto-modulation: auto auto6t auto6c qam



In [42]:
!hdhomerun_config $hdhr_id get /tuner1/status

unable to connect to device


In [35]:
for i in range(4):
    !hdhomerun_config $hdhr_id get /tuner$i/status

ch=8vsb:479000000 lock=8vsb ss=100 snq=100 seq=100 bps=2714720 pps=258
ch=8vsb:473000000 lock=8vsb ss=87 snq=82 seq=100 bps=2687648 pps=267
ch=8vsb:575000000 lock=8vsb ss=100 snq=100 seq=100 bps=6364928 pps=604
ch=8vsb:593000000 lock=8vsb ss=100 snq=100 seq=100 bps=1910080 pps=186


In [37]:
for i in range(4):
    !hdhomerun_config $hdhr_id get /tuner$i/program

6
2
1
4
